In [ ]:
from nntool.api import NNGraph
from PIL import Image
import numpy as np

In [ ]:
G = NNGraph.load_graph("models/vww_96_int8.tflite", load_quantization=True)
G.name = "vww"
G.adjust_order()
G.fusions("scaled_match_group")

In [ ]:
G.quantize(
    graph_options={"use_ne16": True},
)

In [ ]:
G.insert_resizer(
    "input_1", (240, 320), "bilinear"
)
#G.adjust_order()

In [ ]:
G.qshow([G[0]])
G.show([G[0]])

In [ ]:
img = Image.open("images/COCO_val2014_000000174838_1.ppm").resize((320, 240))
#img.show()
input_data = (np.array(img) / 255) #.transpose(2, 0, 1)
input_data.min(), input_data.max()

In [ ]:
fout = G.execute([input_data])
dqout = G.execute([input_data], dequantize=True)
qout = G.execute([input_data], dequantize=False, quantize=True)

In [ ]:
#G.qsnrs(fout, dqout)
qout[-1][0]

In [ ]:
#G.show(show_constants=True)

In [ ]:
#G['model_conv2d_13_Conv2D'].at_options.force_stop_l1_promotion = True
G[0].allocate = True
res = G.execute_on_target(
    input_tensors=qout[0],
    directory="/tmp/test_vww",
    settings={
        "tensor_directory": "at_model/tensors",
        "model_directory": "at_model",
        'l1_size': 128000,
        'l2_size': 1300000,
        'l3_flash_device': 'AT_MEM_L3_MRAMFLASH',
        'cluster_stack_size': 1024,
        'cluster_slave_stack_size': 512,
        'graph_l1_promotion': 1,
    },
    at_loglevel=2,
    memory=True,
    print_output=True
)

In [ ]:
G.qsnrs(qout, res.output_tensors)

In [ ]:
Image.fromarray(res.output_tensors[2][0]).show()
Image.fromarray(qout[2][0]).show()

In [ ]:
%matplotlib widget
import matplotlib
matplotlib.rcParams.update({'font.size': 12})
res.plot_memory_boxes(figsize=(10,15))

In [ ]:
G.draw()